In [1]:
# Importing required libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load the data
df = pd.read_csv('../Dataset/classificationData.csv')
print("The dataset has {} rows and {} columns.".format(df.shape[0], df.shape[1]))
df = df.drop(df.columns[0], axis=1) # Drop first column

del df['Crime Occurences']

df.head(3)

The dataset has 20160 rows and 7 columns.


,district,day,month,hour,Class
0,SOUTHERN,Wednesday,1,18,Extreme
1,SOUTHERN,Wednesday,3,18,Extreme
2,SOUTHERN,Friday,8,18,Extreme


In [3]:
from sklearn.preprocessing import LabelEncoder
y = df.iloc[:, -1].values
le = LabelEncoder()
y = le.fit_transform(y)

In [4]:
%%capture
df = pd.get_dummies(df, columns=['district', 'day'], drop_first=True)
print(df.head())

X = df.loc[:, df.columns != 'Class'].values;

In [5]:
# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 1)

In [6]:
# Importing classification packages.
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

SEED = 1 # Random Seed

# Classification
lr = LogisticRegression(random_state=SEED)
knn = KNN()
xg = XGBClassifier()
dt = DecisionTreeClassifier(random_state=SEED)
rf = RandomForestClassifier(random_state=SEED)

# Define a list called classifier that contains the tuples (classifier_name, classifier)
classifiers = [('Logistic Regression', lr),
               ('K Nearest Neighbours', knn),
               ('Classification Tree', dt),
               ('Forest', rf),
               ('XGB', xg)]

In [7]:
# Iterating over the defined list of tuples containing the classifiers
for clf_name, clf in classifiers:
    # Fitting
    clf.fit(X_train, y_train)
    # Predicting
    y_pred = clf.predict(X_test)
    
    print('{:s} : {:s}'.format(clf_name, classification_report(y_test, y_pred)))

Logistic Regression :               precision    recall  f1-score   support

           0       0.66      0.70      0.68       517
           1       0.55      0.63      0.59       478
           2       0.66      0.80      0.72       512
           3       0.76      0.44      0.56       509

    accuracy                           0.64      2016
   macro avg       0.66      0.64      0.64      2016
weighted avg       0.66      0.64      0.64      2016

K Nearest Neighbours :               precision    recall  f1-score   support

           0       0.79      0.84      0.82       517
           1       0.62      0.64      0.63       478
           2       0.83      0.88      0.86       512
           3       0.71      0.60      0.65       509

    accuracy                           0.74      2016
   macro avg       0.74      0.74      0.74      2016
weighted avg       0.74      0.74      0.74      2016

Classification Tree :               precision    recall  f1-score   support

        